In [18]:
# %matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Angle
from astropy.time import Time
from astropy import units as u
import tqdm
import random
import string
from astropy.visualization import astropy_mpl_style, quantity_support

In this noteboook I keep the old codes that is necesary to rule out for different reasons, or for try different options with other codes

In [19]:
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import astropy.units as u
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def DeltaTime(Date_i, Date_f, t_scale):
    """Calcula el delta de tiempo entre dos fechas con diferentes escalas."""
    scale = t_scale[0]
    sc = t_scale[1]

    a = Time(Date_i, format='iso', scale='utc').datetime.hour - 24
    b = Time(Date_f, format='iso', scale='utc').datetime.hour

    if scale == 's':
        length = int((b - a) / (sc / 3600))
        t_ = np.linspace(a, b, length)
    elif scale == 'm':
        length = int((b - a) / (sc / 60))
        t_ = np.linspace(a, b, length)
    elif scale == 'h':
        length = int((b - a) / sc)
        t_ = np.linspace(a, b, length)
    
    return t_

def CreateTime(Date_i, Date_f, t_scale):
    """Crea un array de tiempo alrededor de la medianoche."""
    time_midnight = Time(Time(Date_i, format='iso', scale='utc').iso.split()[0] + ' 00:00:00', 
                        format='iso', scale='utc')
    delta = DeltaTime(Date_i, Date_f, t_scale) * u.hour
    return time_midnight + delta

def process_time_slot(each_time, alert, observer, limit):
    """Procesa un slot de tiempo para calcular coordenadas Alt/Az."""
    celestial_coord = SkyCoord(ra=alert['RA'], dec=alert['DEC'])
    altaz_coord = celestial_coord.transform_to(AltAz(obstime=each_time, location=observer))
    
    state = altaz_coord.alt > limit * u.deg
    result = alert.copy()
    result['Observable'] = state
    result['Az'] = altaz_coord.az.deg
    result['Alt'] = altaz_coord.alt.deg
    result['Time'] = each_time
    return result

def Observations(observer, alert, Date_i, Date_f, time_scale, rango, limit, m_min):
    """Retorna las observaciones clasificadas usando paralelización."""
    time = CreateTime(Date_i, Date_f, time_scale)
    lat_conv, lon_conv = observer
    observer_loc = EarthLocation(lat=lat_conv * u.deg, lon=lon_conv * u.deg)
    
    # Pre-procesamiento de alertas
    alert = alert.copy()
    alert['Label'] = range(1, rango)
    alert = alert[alert['Mag'] <= m_min].copy()
    
    # Paralelización del cálculo de coordenadas
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_time_slot, each_time, alert, observer_loc, limit) 
                  for each_time in time]
        
        Big_Data = [future.result() for future in as_completed(futures)]
    
    return pd.concat(Big_Data, axis=0)

def ExpositionTime(K, m, m_ref):
    """Calcula el tiempo de exposición."""
    return K * (10 ** (0.4 * (m - m_ref)))

def process_target(label, Data, Time, General_time, t_expo, K, m_ref):
    """Procesa un objetivo individual para determinar su observabilidad."""
    Target = Data.loc[Data['Label'] == label].copy()
    if Target.empty:
        return None
    
    Target['Time'] = Time
    Target['Time expo'] = ExpositionTime(K, Target['Mag'], m_ref)
    Target = Target[(Target['Observable'] != False) & (Target['Time expo'] <= t_expo)]
    return Target if not Target.empty else None

def OrderLimits(Data, rango, Time, General_time, t_expo, K, m_ref):
    """Organiza los límites de observación usando paralelización."""
    # Procesamiento paralelo de objetivos
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_target, i, Data, Time, General_time, t_expo, K, m_ref): i 
                  for i in range(1, rango)}
        
        targets = []
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                targets.append(result)
    
    if not targets:
        return [], pd.DataFrame()
    
    # Concatenación de resultados
    obs = pd.concat(targets, axis=0).reset_index(drop=True).drop(['Observable'], axis=1)
    
    # Ordenamiento secuencial (no paralelizable debido a dependencias temporales)
    order = []
    first_target = obs[obs['Time'] == Time[0]]
    
    if not first_target.empty:
        first_target = first_target.sort_values(by='Alt', ascending=False, na_position='first').head(1)
        first_target['Time'] = General_time[0]
        order.append(first_target)
        
        time_flag = General_time[0] + first_target['Time expo'].iloc[0] * u.second
        position_flag = first_target
    else:
        time_flag = General_time[0]
        position_flag = pd.DataFrame()

    for i in range(1, len(Time)):
        priority = obs.loc[obs['Time'] == Time[i]].sort_values('Alt', ascending=False, na_position='first').head(1)
        current_time = General_time[i]

        if priority.empty:
            continue

        priority['Time'] = current_time

        if not priority.empty:
            if time_flag >= current_time:
                if not position_flag.empty and position_flag['Label'].iloc[0] == priority['Label'].iloc[0]:
                    order.append(priority)
                else:
                    position_flag['Time'] = current_time
                    order.append(position_flag)
            else:
                order.append(priority)
                time_flag = current_time + priority['Time expo'].iloc[0] * u.second
                position_flag = priority

    order = pd.concat(order, axis=0).reset_index(drop=True).drop(['Az', 'Alt'], axis=1)
    return targets, order

In [20]:
#Playtime data

#Tuple in order of (latitude,longitude) in degrees
observer = 4.609 , -74.081
 
#Slot of time and date
date_i = '2025-03-13 19:00:00'
date_f = '2025-03-14 07:00:00'

#Time scale of the observation, the minimum is 5 seconds
timescale = ['m', 30]

#Limit of telescopes for good observations, usually are 33 degree on altitude
limit = 33 

# Limit on magnitude for the specific telescope
m_min = 22

# Parameter K and m_ref, random numbers until i define the units
K = 10
m_ref = -21 #The sun magnitude

#Time exposition minimum for the telescope
t_expo = 1.5e-5

#RANDOM DATA
def random_data(n):
    #random names
    nombres = [''.join(random.choices(string.ascii_uppercase, k=random.randint(5, 7))) for _ in range(n)]
    
    #random ra
    ra = [f"{random.randint(0, 23):02d}h{random.randint(0, 59):02d}m{random.randint(0, 59):02d}.{random.randint(0, 99):02d}s" 
          for _ in range(n)]
    
    #random dec
    dec = [f"{random.choice(['+', '-'])}{random.randint(0, 89):02d}d{random.randint(0, 59):02d}m{random.randint(0, 59):02d}.{random.randint(0, 90):02d}s" 
           for _ in range(n)]
    
    #random magnitude
    mag = [random.randint(-27, 35) for _ in range(n)]
    
    #Create the dataframe
    df = pd.DataFrame({
        'Name': nombres,
        'RA': ra,
        'DEC': dec,
        'Mag': mag})
    
    return df


In [26]:
#Dataframe with random targets
alerts = random_data(10)

#Amount of targets, this is for the labels of graphic
many = len(alerts) + 1

#Observations with the data
Data = Observations(observer, alerts, date_i, date_f,timescale,many,limit,m_min)
time = DeltaTime(date_i,date_f,timescale)
general_time = CreateTime(date_i,date_f,timescale)
graphic,order = OrderLimits(Data,many,time,general_time,t_expo,K,m_ref)

print('Order for observation')
order

Order for observation


""
